## Лекция №2

### Восстановление пропущенных значений

В таблице могут быть пустые ячейки (с пропущенными значениями) и/или с некорректными значениями.

In [1]:
# Необходимые пакеты
import numpy as np
import pandas as pd
import math

In [2]:
df = pd.read_csv('./data/table_0201.csv')
df

,Студент,Пол,Рост,Вес,Место на олимпиаде,Город
0,Иванов,1.0,172.0,107.0,3.0,NaN
1,Запеканка,NaN,185.0,71.0,-4.0,NaN
2,Ватрушкина,0.0,168.0,666.0,2.0,NaN
3,Ололоева,0.0,NaN,85.0,1.0,NaN
4,Сидорова,0.0,165.0,15.0,2.0,NaN
5,Петров,1.0,172.0,71.0,4.0,NaN
6,Алексеева,NaN,NaN,NaN,NaN,NaN
7,Андреев,1.0,NaN,NaN,3.0,NaN
8,Новикова,0.0,NaN,53.0,4.0,NaN


#### Что можно/нужно сделать?

Если пропущенных значений много -> удалить объект (строку)

In [3]:
df_new = df.drop([6])
df_new

,Студент,Пол,Рост,Вес,Место на олимпиаде,Город
0,Иванов,1.0,172.0,107.0,3.0,NaN
1,Запеканка,NaN,185.0,71.0,-4.0,NaN
2,Ватрушкина,0.0,168.0,666.0,2.0,NaN
3,Ололоева,0.0,NaN,85.0,1.0,NaN
4,Сидорова,0.0,165.0,15.0,2.0,NaN
5,Петров,1.0,172.0,71.0,4.0,NaN
7,Андреев,1.0,NaN,NaN,3.0,NaN
8,Новикова,0.0,NaN,53.0,4.0,NaN


Чтобы удалить строку в "оригинальной" таблице, необходимо удалять с параметром inplace=True

In [4]:
df.drop([6], inplace=True)
df

,Студент,Пол,Рост,Вес,Место на олимпиаде,Город
0,Иванов,1.0,172.0,107.0,3.0,NaN
1,Запеканка,NaN,185.0,71.0,-4.0,NaN
2,Ватрушкина,0.0,168.0,666.0,2.0,NaN
3,Ололоева,0.0,NaN,85.0,1.0,NaN
4,Сидорова,0.0,165.0,15.0,2.0,NaN
5,Петров,1.0,172.0,71.0,4.0,NaN
7,Андреев,1.0,NaN,NaN,3.0,NaN
8,Новикова,0.0,NaN,53.0,4.0,NaN


Удалим объекты (строки), где количество значимых (не NaN) признаков меньше 4-х

In [5]:
df.dropna(thresh=4, inplace=True)   # thresh=4, т.е. не меньше 4-х. Удалим объект Андреев (количество значимых признаков == 3)
df

,Студент,Пол,Рост,Вес,Место на олимпиаде,Город
0,Иванов,1.0,172.0,107.0,3.0,NaN
1,Запеканка,NaN,185.0,71.0,-4.0,NaN
2,Ватрушкина,0.0,168.0,666.0,2.0,NaN
3,Ололоева,0.0,NaN,85.0,1.0,NaN
4,Сидорова,0.0,165.0,15.0,2.0,NaN
5,Петров,1.0,172.0,71.0,4.0,NaN
8,Новикова,0.0,NaN,53.0,4.0,NaN


Как мы видим признак (столбец) "Город" не несет никакой "смысловой нагрузки". Его надо удалить.

In [6]:
df.dropna(thresh=1, axis="columns", inplace=True)    # или axis=1 - это столбцы, axis=0 - это строки (по умолчанию)
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,NaN,185.0,71.0,-4.0
2,Ватрушкина,0.0,168.0,666.0,2.0
3,Ололоева,0.0,NaN,85.0,1.0
4,Сидорова,0.0,165.0,15.0,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,NaN,53.0,4.0


Поправим некорректные значения. Вес 666 кг это уже слишком (а 15 кг слишком маленький). И место не может быть меньше < 1.

In [7]:
df.loc[(df['Вес'] > 200) | (df['Вес'] < 20), 'Вес'] = None

In [8]:
df.loc[(df['Место на олимпиаде'] < 1), 'Место на олимпиаде'] = None
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,NaN,185.0,71.0,NaN
2,Ватрушкина,0.0,168.0,NaN,2.0
3,Ололоева,0.0,NaN,85.0,1.0
4,Сидорова,0.0,165.0,NaN,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,NaN,53.0,4.0


Теперь можно заняться восстановлением "пропущенных" значений (NaN). Вот некоторые варианты замены:
- Среднее арифметическое значение
- Медиана
- Мода

Заменим пропущенные значения в признаке "Рост" на среднее арифметическое признака.

In [9]:
df['Рост'].fillna(df['Рост'].mean(), inplace=True) # ! Внимание. Среднее чувствительно к выбросам в данных
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,NaN,185.0,71.0,NaN
2,Ватрушкина,0.0,168.0,NaN,2.0
3,Ололоева,0.0,172.4,85.0,1.0
4,Сидорова,0.0,165.0,NaN,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,172.4,53.0,4.0


Пропущенные значения в признаке "Вес" заменим на моду (самое часто встречающее значение == 71)

In [10]:
df['Вес'].fillna(df['Вес'].mode()[0], inplace=True) # mode()[0] - первая мода, т.к. их может быть несколько
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,NaN,185.0,71.0,NaN
2,Ватрушкина,0.0,168.0,71.0,2.0
3,Ололоева,0.0,172.4,85.0,1.0
4,Сидорова,0.0,165.0,71.0,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,172.4,53.0,4.0


Для места на олимпиаде выберем медиану.

In [11]:
# ! Медиану округлим до целого, т.к. место может быть только целым )))
df['Место на олимпиаде'].fillna(round(df['Место на олимпиаде'].median()), inplace=True)
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,NaN,185.0,71.0,2.0
2,Ватрушкина,0.0,168.0,71.0,2.0
3,Ололоева,0.0,172.4,85.0,1.0
4,Сидорова,0.0,165.0,71.0,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,172.4,53.0,4.0


Усложним чуть-чуть задачу. Признак "Пол" заполним значением с определенной вероятностью.
Значение 1 (мужской пол), всего 2 из 6, итого вероятность = 2/6.
Значение 0 (женский пол), всего 4 из 6, итого вероятность = 4/6

In [12]:
count_gender = df['Пол'].count()      # всего значений в признаке "Пол" (без NaN)

In [13]:
male_weight = len(df[df['Пол'] == 1]) / count_gender      # вероятность для "мужского пола"

In [14]:
female_weight = len(df[df['Пол'] == 0]) / count_gender    # вероятность для "женского пола"

In [15]:
import random
df['Пол'].fillna(random.choices([1, 0], weights=[male_weight, female_weight])[0], inplace=True)
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,1.0,185.0,71.0,2.0
2,Ватрушкина,0.0,168.0,71.0,2.0
3,Ололоева,0.0,172.4,85.0,1.0
4,Сидорова,0.0,165.0,71.0,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,172.4,53.0,4.0


### Метрики

#### 1. Евклидова метрика

$p(P,Q) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2 + (p_3 - q_3)^2 \ldots + (p_n - q_n)^2}$

#### 2. Метрика Манхеттен

$p(P,Q) = |p_1 - q_1| + |p_2 - q_2| +\ldots+ |p_n - q_n|$

#### 3. max - метрика

$p(P,Q) = max\{|p_1 - q_1|, |p_2 - q_2|,\cdots, |p_n - q_n|\}$

***Пример***

Пусть дан набор данных с пропущенными значениями:

In [16]:
df = pd.read_csv('./data/table_0202.csv')
df

,objects,P1,P2,P3,P4,P
0,A1,3,4,5,3,4.0
1,A2,5,5,5,4,3.0
2,A3,4,3,3,2,5.0
3,A,5,4,3,3,NaN


В столбце P, мы видим пропущенное значение. Попробуем восстановить его с помощью метрик. В рассчетах столбец (признак) P, пока участие не принимает. 

Рассчитаем метрики:

In [17]:
# Все операции векторизированны
euclid_A_A1 = np.sqrt(np.sum((df.iloc[0][1:5] - df.iloc[3][1:5])**2))
euclid_A_A2 = np.sqrt(np.sum((df.iloc[1][1:5] - df.iloc[3][1:5])**2))
euclid_A_A3 = np.sqrt(np.sum((df.iloc[2][1:5] - df.iloc[3][1:5])**2))

In [18]:
manhattan_A_A1 = np.sum(np.abs(df.iloc[0][1:5] - df.iloc[3][1:5]))
manhattan_A_A2 = np.sum(np.abs(df.iloc[1][1:5] - df.iloc[3][1:5]))
manhattan_A_A3 = np.sum(np.abs(df.iloc[2][1:5] - df.iloc[3][1:5]))

In [19]:
max_A_A1 = np.max(np.abs(df.iloc[0][1:5] - df.iloc[3][1:5]))
max_A_A2 = np.max(np.abs(df.iloc[1][1:5] - df.iloc[3][1:5]))
max_A_A3 = np.max(np.abs(df.iloc[2][1:5] - df.iloc[3][1:5]))

In [20]:
metric_dict = {'От A до A1': np.round([euclid_A_A1, manhattan_A_A1, max_A_A1], decimals=2),  # округлить до 2-х знаков
               'От A до A2': np.round([euclid_A_A2, manhattan_A_A2, max_A_A2], decimals=2),
               'От A до A3': np.round([euclid_A_A3, manhattan_A_A3, max_A_A3], decimals=2)}

In [21]:
df_metric = pd.DataFrame(metric_dict, index=['Евклид', 'Манхеттен', 'Макс'])
df_metric

,От A до A1,От A до A2,От A до A3
Евклид,2.83,2.45,1.73
Манхеттен,4.00,4.00,3.00
Макс,2.00,2.00,1.00


Получилось. Но что делать дальше с этими числами?<br>Нужно взять их линейную комбинацию со значениями признака P.

In [22]:
df['P'][0:3]

0    4.0
1    3.0
2    5.0
Name: P, dtype: float64

***Евклидова метрика***

${\Large\frac{1}{\frac{1}{2.83} + \frac{1}{2.45} + \frac{1}{1.73}} \cdot \left(\frac{4}{2.83} + \frac{3}{2.45} + \frac{5}{1.73}\right) = 4.13}$

In [23]:
(1 / np.sum(1 / df_metric.iloc[0])) *  np.sum(np.array(df['P'][0:3]) / df_metric.iloc[0])

4.126811842244475

### Корреляция Пирсона

In [24]:
df_c = pd.read_csv('./data/table_0204.csv')
df_c

,Грудь,Талия,Бёдра,Рост,Вес
Ж1,99.0,56,91,160,58
Ж2,89.0,58,89,157,48
Ж3,91.0,64,91,165,54
Ж4,91.0,51,91,170,54
Ж5,86.0,56,84,157,44
Ж6,97.0,53,86,175,56
АЖ,NaN,51,91,165,54


In [25]:
df_c.dropna().corr()     # Встроенная функция в pandas (по умолчанию используется метод Пирсона)

,Грудь,Талия,Бёдра,Рост,Вес
Грудь,1.000000,-0.219903,0.342287,0.457826,0.906844
Талия,-0.219903,1.000000,0.231149,-0.415612,-0.123432
Бёдра,0.342287,0.231149,1.000000,0.072044,0.612344
Рост,0.457826,-0.415612,0.072044,1.000000,0.606136
Вес,0.906844,-0.123432,0.612344,0.606136,1.000000


**Вычисление корреляции по этапам**

1. Сумма всех значений X и Y: $\sum{x_i}$ , $\sum{y_i}$

In [26]:
sum_x = np.sum(df_c['Грудь'][0:6])      # без последней строки
sum_y = np.sum(df_c['Вес'][0:6])
sum_x, sum_y

(553.0, 314)

2. Сумма квадратов X и Y: $\sum{x^2}$ , $\sum{y^2}$

In [27]:
sum_sq_x = np.sum(df_c['Грудь'][0:6] ** 2)
sum_sq_y = np.sum(df_c['Вес'][0:6] ** 2)
sum_sq_x, sum_sq_y

(51089.0, 16572)

3. Сумма всех произведений значений X и Y: $\sum{(x_i \times y_i})$

In [28]:
sum_prod_xy = np.sum(df_c['Грудь'][0:6] * df_c['Вес'][0:6])
sum_prod_xy

29058.0

*Количество элементов*

In [29]:
n_xy = df_c['Грудь'].count()      # Количество значений (без NaN)
n_xy

6

4. Подставляем в формулу:

${\Large r_{xy} = \frac{(n\_xy \; \times \; sum\_prod\_xy) - (sum\_x \; \times \; sum\_y)}{\sqrt{ \left [(n\_xy \; \times \; sum\_sq\_x) \; - \; sum\_x^2 \right] \cdot \left [(n\_xy \; \times \; sum\_sq\_y) \; - \; sum\_y^2 \right] }   }}$

In [30]:
r_xy = ((n_xy * sum_prod_xy) -(sum_x * sum_y)) / math.sqrt(((n_xy * sum_sq_x) - sum_x**2) * ((n_xy * sum_sq_y) - sum_y**2)) 
r_xy

0.9068439481043561

Как видим, рассчитаный коэффициент корреляции 'Грудь' к 'Вес' соответствует рассчитаному функцией pandas.

In [31]:
df_c.corr()['Грудь']['Вес']

0.906843948104356

#### Восстановим пропущенное значение признака 'Грудь' по формуле:

${\Large P(A) = \bar P + \frac {\sum _{j=1}^n r(P, P_i)(P_i(A) - \bar P_i) } {\sum _{i=1}^m |r(P, P_i)|} }$

1. Сумма произведений коэффициентов корреляции для признаков на разницу признаков других объектов и среднего других признаков.<br>
2. Деленная на сумму модулей коэффициентов корреляции.
3. Плюс среднее признака с пропущенным значением

**! Внимание ! Коэффициенты корреляции и среднее вычисляется без последней строки (без объекта для которого восстанавливаем значение). Само собой количество признаков и средних будет без признака для которого восстанавливаем значение (без признака "Грудь" и его среднего)**

In [32]:
features_obj = df_c.loc['АЖ'][1:]   # Значимые (со значением, без NaN) признаки искомого (для которого будем искать) объекта АЖ 
features_obj                        # loc[] - взять строку

Талия     51.0
Бёдра     91.0
Рост     165.0
Вес       54.0
Name: АЖ, dtype: float64

In [33]:
# Коэффициенты корреляции для признака "Грудь" (первое значение пропускаем, т.к. там "Грудь" к "Груди")
corr_breast = df_c.corr()['Грудь'][1:]    
corr_breast

Талия   -0.219903
Бёдра    0.342287
Рост     0.457826
Вес      0.906844
Name: Грудь, dtype: float64

In [34]:
mean_f = df_c.dropna().mean()      # средние признаков, без последней строки с NaN значением
mean_f

Грудь     92.166667
Талия     56.333333
Бёдра     88.666667
Рост     164.000000
Вес       52.333333
dtype: float64

In [35]:
mean_f.iloc[1:]     # Средние не включая признака "Грудь"

Талия     56.333333
Бёдра     88.666667
Рост     164.000000
Вес       52.333333
dtype: float64

In [36]:
# 1. Вычислим сумму модулей коэффициентов корреляции признака "Грудь"
sum_corr = np.sum(np.abs(corr_breast))

In [37]:
# 2. Вычисляем пропущенное значение для признака "Грудь"
res_feature = mean_f['Грудь'] + np.sum(corr_breast * (features_obj - mean_f.iloc[1:])) / sum_corr
res_feature

94.2118178524154

In [38]:
ser_01 = pd.Series([0, 1, 2])
ser_01

0    0
1    1
2    2
dtype: int64

In [39]:
ser_02 = pd.Series([2, 1, 0])

In [40]:
np.sqrt(np.sum((ser_01 -ser_02)**2))

2.8284271247461903

In [41]:
np.sum(np.abs(ser_01 - ser_02))

4

In [42]:
np.max(np.abs(ser_01 - ser_02))

2

In [43]:
p = pd.Series([1,0,5,2,2])

In [44]:
min_p = min(p)
max_p = max(p)
min_p, max_p

(0, 5)

In [45]:
norm_p = (p - min_p) / (max_p - min_p)
norm_p

0    0.2
1    0.0
2    1.0
3    0.4
4    0.4
dtype: float64

In [46]:
mean_p = p.mean()
std_p = p.std()
mean_p, std_p

(2.0, 1.8708286933869707)

In [47]:
(p - mean_p) / (std_p)

0   -0.534522
1   -1.069045
2    1.603567
3    0.000000
4    0.000000
dtype: float64

### Задание: Восстановление значения с помощью метрики Манхеттен

_TODO: Написать функцию_

In [68]:
row_01 = [5, 5, 5, 3]
row_02 = [5, 3, 4, 4]
row_03 = [2, 5, 3, 5]
row_04 = [3, 4, 4, np.nan]

In [70]:
test_df = pd.DataFrame(index=['Вася', 'Петя', 'Маша', 'Саша'], columns=['Молчание ягнят', 'Титаник', 'Матрица', 'Гарри Потер'])
test_df.loc['Вася'] = row_01
test_df.loc['Петя'] = row_02
test_df.loc['Маша'] = row_03
test_df.loc['Саша'] = row_04
test_df

,Молчание ягнят,Титаник,Матрица,Гарри Потер
Вася,5,5,5,3
Петя,5,3,4,4
Маша,2,5,3,5
Саша,3,4,4,NaN


In [72]:
test_df.iloc[0][0:3]

Молчание ягнят    5
Титаник           5
Матрица           5
Name: Вася, dtype: object

Рассчитаем Манхеттен метрику

In [77]:
m_01 = np.sum(np.abs(test_df.iloc[0][0:3] - test_df.iloc[3][0:3]))
m_02 = np.sum(np.abs(test_df.iloc[1][0:3] - test_df.iloc[3][0:3]))
m_03 = np.sum(np.abs(test_df.iloc[2][0:3] - test_df.iloc[3][0:3]))
m_all = pd.Series([m_01, m_02, m_03])
m_all

0    4
1    3
2    3
dtype: int64

Восстановим значение

In [76]:
test_df['Гарри Потер'][0:3]

Вася    3
Петя    4
Маша    5
Name: Гарри Потер, dtype: object

In [80]:
test_df.fillna((1 / np.sum(1 / m_all)) *  np.sum(np.array(test_df['Гарри Потер'][0:3]) / m_all), inplace=True)
test_df

,Молчание ягнят,Титаник,Матрица,Гарри Потер
Вася,5,5,5,3.000000
Петя,5,3,4,4.000000
Маша,2,5,3,5.000000
Саша,3,4,4,4.090909


In [87]:
row_A = [1, 0, 1, 0, 1, 0]
row_C = [1, 1, 0, 1, 1, 0]
test_anon = pd.DataFrame(index=['A', 'C'], columns=range(1, 7))
test_anon.loc['A'] = row_A
test_anon.loc['C'] = row_C
test_anon

,1,2,3,4,5,6
A,1,0,1,0,1,0
C,1,1,0,1,1,0


In [89]:
euc = np.sqrt(np.sum((test_anon.loc['A'] - test_anon.loc['C'])**2))
euc

1.7320508075688772